In [51]:
import lifelib
import timeit
import pandas as pd
import numpy as np
import modelx as mx
import openpyxl

ex4 = mx.read_model('CashValue_ME_EX4')
Projection = ex4.Projection

timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=5)

0.7503534269999363

In [52]:
# Projection.model_point_table = Projection.model_point_1
table = Projection.model_point_table
# print("Number of model points: ", len(table))
# print("Model points: ", table)
# points = Projection.model_point_table_ext()
# points = Projection.model_point()["scen_id"].values[990:1010]
points = Projection.model_point()
print("model_point(): ", points)
print("with indices: ", points.index)

model_point():                   spec_id  age_at_entry sex  policy_term  policy_count  \
point_id scen_id                                                        
1        1             A            20   M           10           100   
         2             A            20   M           10           100   
         3             A            20   M           10           100   
         4             A            20   M           10           100   
         5             A            20   M           10           100   
...                  ...           ...  ..          ...           ...   
9        996           A            20   M           10           100   
         997           A            20   M           10           100   
         998           A            20   M           10           100   
         999           A            20   M           10           100   
         1000          A            20   M           10           100   

                  sum_assured  dur

In [53]:
pols = ex4.Projection.pols_if_at(12, "BEF_DECR")
print(np.shape(pols))
print(sum(pols))
print(pols)

(9000,)
900000.0
[100. 100. 100. ... 100. 100. 100.]


In [54]:
Projection.pols_if(1)

array([100., 100., 100., ..., 100., 100., 100.])

In [56]:
Projection.result_pv()['Net Cashflow']

point_id  scen_id
1         1         -3.475896e+06
          2         -3.475896e+06
          3         -3.475896e+06
          4         -3.475896e+06
          5         -3.475896e+06
                         ...     
9         996       -1.490894e+07
          997       -1.245624e+07
          998       -1.196305e+07
          999       -8.771397e+06
          1000      -1.148598e+07
Name: Net Cashflow, Length: 9000, dtype: float64

In [17]:
print(ex4.Projection.max_proj_len())
ex4.Projection.pv_premiums()

121


array([50000000., 50000000., 50000000., ..., 30000000., 30000000.,
       30000000.])

In [18]:
ex4.Projection.pols_new_biz(0)

array([100, 100, 100, ..., 100, 100, 100])

In [45]:
inv = Projection.inv_return_mth(2)
print("Montlhy investment returns: ", inv)
print("with shape: ", np.shape(inv))

Montlhy investment returns:  [ 0.00807793 -0.00048898 -0.00302246 ... -0.00917993 -0.00629737
 -0.00596671]
with shape:  (9000,)


In [ ]:
Projection.result_pv()